##**DATA CHALLENGE LATAM**`

####**QUESTION 1:** Obtener los top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.

**QUESTION 1 - MEMORY:**

In [ ]:
import pandas as pd
from typing import List, Tuple
from datetime import datetime

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
resultado = []

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    # Leer el archivo JSON de un dataframe de pandas
    df = pd.read_json(file_path, lines=True)

    # Convertimos la columna de fechas al formato deseado
    df['date'] = pd.to_datetime(df['date'])
    df['date'] = df['date'] + pd.Timedelta(hours=-5)# Convertir a la zona horaria deseada gtm - 5 Peru

    # Agregar una columna de fecha formateada
    df['formatted_date'] = df['date'].dt.date

    # Calcular el recuento de publicaciones por fecha y por usuario
    pub_por_fecha = df['formatted_date'].value_counts()
    df['username'] = df['user'].apply(lambda x: x['username'])# Usamos lambda a cada elemento de la columna 'user' para tomar cada diccionario x y extrae el valor asociado a la clave 'username'.
    pub_usuario_fecha = df.groupby(['formatted_date', 'username']).size().unstack(fill_value=0) # pivotear los nombres a columnas y contar ocurrencias, llenar con 0 si no hay ocurrencias

    # Obtener las top 10 fechas con más publicaciones
    top_fechas = pub_por_fecha.head(10).index.tolist() # Obtenemos los top10 de los indices y lo convertimos en una lista

    # Obtener el usuario con más publicaciones para cada una de las top 10 fechas
    for fecha in top_fechas:
        usuario_mas_publicaciones = pub_usuario_fecha.loc[fecha].idxmax()
        resultado.append((fecha, usuario_mas_publicaciones))
    return resultado

resultado = q1_memory(file_path)
print(resultado)


**QUESTION 1 - TIME:**

In [ ]:
from typing import List, Tuple
from datetime import timedelta,datetime
from collections import defaultdict
import json

## OBJETIVO 1 ##
## Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días.

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
resultado=[]#Lista para concatenar las tuplas (fecha,usuario mas publicaciones)
pub_por_fecha = defaultdict(int) # Diccionario para almacenar el recuento de publicaciones por fecha
pub_usuario_fecha = defaultdict(lambda: defaultdict(int))# Diccionario para almacenar el recuento de publicaciones por usuario para cada fecha

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    #Leer el contenido del fichero JSON
    with open(file_path) as contenido:  #type contenido = class '_io.TextIOWrapper'
        lineas = contenido.readlines()  #type lineas = class 'list' y contiene \n de cada linea
    lista_tweets = [json.loads(linea.rstrip()) for linea in lineas] #recorremos la lista contenido y quitamos los espacios de la derecha que pueda tener la linea y la incluimos dentro de una lista, con esto evitamos los \n de cada linea

    #Recorremos la lista de tweets
    for tweet in lista_tweets:
        #Obtener el contenido del diccionario con clave date
        fecha_str = tweet.get("date")
        #Convertir el GMT al pais que desee analizar o mantener en GMT = 0 como viene por defecto
        fecha_hora = datetime.fromisoformat(fecha_str)
        nueva_fecha_hora = fecha_hora + timedelta(hours=-5) #en nuestro modificamos al gmt de peru -5,o de acuerdo a lo requerido analizar.
        fecha_formateada = nueva_fecha_hora.strftime("%Y-%m-%d") #Formateamos la fecha al formato indicado
        pub_por_fecha[fecha_formateada] += 1
        pub_usuario_fecha[fecha_formateada][tweet['user']['username']] +=1

    # Obtener las top 10 fechas con más publicaciones
    top_fechas = sorted(pub_por_fecha.items(), key=lambda x: x[1], reverse=True)[:10]
    # Obtener el usuario con más publicaciones para cada una de las top 10 fechas
    for fecha, _ in top_fechas:#solo iteramos por el primer valor de la tupla
        """
        pub_usuario_fecha es un dictionario[ fecha ][ dictionario[usuario][conteo_publicaciones] ]
        con la funcion max a los valores de pub_usuario_fecha(es un diccionario) tome la primera fecha del top de la iteracion hasta llegar a la decima fecha
        en el parametro key se pone en base a que valores realizara la funcion max en este caso, asignamos pub_usuario_fecha[fecha].get a key
        para que lo haga en base a los valores de conteo de publicaciones de los usuario
        """
        usuario_mas_publicaciones = max(pub_usuario_fecha[fecha], key=pub_usuario_fecha[fecha].get)
        resultado.append(((datetime.strptime(fecha, "%Y-%m-%d")).date(), usuario_mas_publicaciones))# cada resultado de la fecha, usuario de mas publiciones se agregan en forma de tupla a la lista resultado

    return resultado

resultado = q1_time(file_path)
print(resultado)

####**QUESTION 2:** Obtener los top 10 emojis más usados con su respectivo conteo.

**QUESTION 2 - MEMORY:**

In [ ]:
from typing import List, Tuple
import json
from collections import defaultdict
import re
import pandas as pd

## OBJETIVO 2 ##
## Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
emojis = defaultdict(int)
patron_sep = r"\ud8" #patron para separar el contenido de cada tweet con los inicios del codigo de emoji que inician con ud8...
patron_regex = r"\\ud8\w{2}\\ud\w{3}" #patron para validar que se trata de un emoji
diccionario = {}

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargar los datos en un dataFrame de Pandas
    df = pd.read_json(file_path, lines=True)
    # Iterar sobre cada tweet en el dataFrame
    for content in df['content']:
        # Buscar emojis del mensaje
        for parttwo in json.dumps(content).split(patron_sep):#Recorreriendo en las divisiones de patron de separador
            if parttwo is not None:
                emoji = patron_sep + parttwo[:8]#Agregamos el patron que se perdio al dividir en la sentencia de arriba y solo consideramos los 8 primero caracteres de segunda parte del codigo del emoji
                if re.search(patron_regex, emoji):#Validamos que cumpla el patron de un codigo de Emoji
                    emojis[emoji] += 1#Agregamos una coincidencia mas al diccionario con clave de codigo del emoji

    # Ordenar usuarios por el top 10 recuento de emojis y con items() convierte el diccionario en una lista de tuplas
    top_emojis = sorted(emojis.items(), key=lambda x: x[1], reverse=True)[:10]

    #Realizamos las lineas de abajo para contener la información en un diccionario y aplicarle json dumps para convertir en cadena para lograr representar los iconos
    for i, (emoji, cantidad) in enumerate(top_emojis, 1):#
        diccionario[f"top{i}"] = emoji
        diccionario[f"vtop{i}"] = cantidad

    diccionario_s = json.dumps(diccionario)#Convertir el dicionario en string
    diccionario_s= diccionario_s.replace("\\\\","\\") #Reemplazar el \\ a \ para que pueda representarse el emoji en el print
    diccionario_o = json.loads(diccionario_s) #convertir a diccionario python
    lista_values = list(diccionario_o.values())#convertir dict_value a una lista para poder iterar

    # Convertir una lista  a una lista de tuplas de dos valores
    lista_de_tuplas = [(lista_values[i], lista_values[i+1]) for i in range(0, len(lista_values), 2)]
    return lista_de_tuplas

resultado = q2_memory(file_path)
print(resultado)

**QUESTION 2 - TIME:**

In [ ]:
from typing import List, Tuple
import json
from collections import defaultdict
import re

## OBJETIVO 2 ##
## Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
emojis = defaultdict(int)
patron_sep = r"\ud8"
patron_regex = r"\\ud8\w{2}\\ud\w{3}"
diccionario = {}

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    with open(file_path) as contenido:  #type contenido = class '_io.TextIOWrapper'
        lineas = contenido.readlines()  #type lineas = class 'list' y contiene \n de cada linea
    lista_tweets = [json.loads(linea.rstrip()) for linea in lineas] #recorremos la lista contenido y quitamos los espacios de la derecha que pueda tener la linea y la incluimos dentro de una lista, con esto evitamos los \n de cada linea

    for tweet in lista_tweets:#Recorriendo los tweet de la lista de tweets
        content = tweet.get("content")#
        # Buscar emojis del mensaje
        for parttwo in json.dumps(content).split(patron_sep):#Recorreriendo en las divisiones de patron de separador
            if parttwo is not None:
                emoji = patron_sep + parttwo[:8]#Agregamos el patron que se perdio al dividir en la sentencia de arriba y solo consideramos los 8 primero caracteres de segunda parte del codigo del emoji
                if re.search(patron_regex, emoji):#Validamos que cumpla el patron de un codigo de Emoji
                    emojis[emoji] += 1#Agregamos una coincidencia mas al diccionario con clave de codigo del emoji

    # Ordenar usuarios por el top 10 recuento de emojis y con items() convierte el diccionario en una lista de tuplas
    top_emojis = sorted(emojis.items(), key=lambda x: x[1], reverse=True)[:10]
    #Realizamos las lineas de abajo para contener la información en un diccionario y aplicarle json dumps para convertir en cadena para lograr representar los iconos
    for i, (emoji, cantidad) in enumerate(top_emojis, 1):#
        diccionario[f"top{i}"] = emoji
        diccionario[f"vtop{i}"] = cantidad

    diccionario_s = json.dumps(diccionario)#Convertir el dicionario en string
    diccionario_s= diccionario_s.replace("\\\\","\\") #Reemplazar el \\ a \ para que pueda representarse el emoji en el print
    diccionario_o = json.loads(diccionario_s) #convertir a diccionario python
    lista_values = list(diccionario_o.values())#convertir dict_value a una lista para poder iterar

    # Convertir una lista  a una lista de tuplas de dos valores
    lista_de_tuplas = [(lista_values[i], lista_values[i+1]) for i in range(0, len(lista_values), 2)]
    return lista_de_tuplas

resultado = q2_time(file_path)
print(resultado)

####**QUESTION 3:** Obtener los top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno  de ellos.

**QUESTION 3 - MEMORY:**

In [ ]:
from typing import List, Tuple
import pandas as pd
from collections import defaultdict

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
# Crear una lista para almacenar las menciones de usuarios
menciones = defaultdict(int)

def q3_memory(file_path: str) -> List[Tuple[str, int]]:
    # Leer el archivo JSON y cargar los datos en un DataFrame de Pandas
    df = pd.read_json(file_path, lines=True)
    # Iterar sobre cada tweet en el DataFrame
    for content in df['content']:
        # Buscar menciones de usuarios (@username)
        for mencion in content.split():# dividimos la cadena de content por espacios ya que entre mencion y mencion hay un espacio
            if mencion.startswith('@'): #validamos que la cadena empieze con @
                usuario = mencion[1:] #quitamos el @ del usuario
                menciones[usuario] += 1 #Agregamos una coincidencia mas al diccionario con clave de usuario

    top_usuarios = sorted(menciones.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_usuarios

resultado= q3_memory(file_path)
print(resultado)

**QUESTION 3 - TIME:**

In [ ]:
from typing import List, Tuple
import json
from collections import defaultdict

## OBJETIVO 3 ##
## El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno  de ellos.

file_path = 'C:\\Users\\anton\\Downloads\\tweets\\farmers-protest-tweets-2021-2-4.json'
menciones = defaultdict(int)

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    with open(file_path) as contenido:  #type contenido = class '_io.TextIOWrapper'
        lineas = contenido.readlines()  #type lineas = class 'list' y contiene \n de cada linea
    lista_tweets = [json.loads(linea.rstrip()) for linea in lineas] #recorremos la lista contenido y quitamos los espacios de la derecha que pueda tener la linea y la incluimos dentro de una lista, con esto evitamos los \n de cada linea

    for tweet in lista_tweets:#Recorrido de la lista de tweets
        content = tweet.get("content")#Extraemos de cada tweet el diccionario con clave content
        # Buscar menciones de usuarios (@username)
        for mencion in content.split():# dividimos la cadena de content por espacios ya que entre mencion y mencion hay un espacio
            if mencion.startswith('@'): #validamos que la cadena empieze con @
                usuario = mencion[1:]#quitamos el @ del usuario
                menciones[usuario] += 1#Agregamos una coincidencia mas al diccionario con clave de usuario

    # Ordenar usuarios por el top 10 recuento de menciones y con items() convierte el diccionario en una lista de tuplas
    top_usuarios = sorted(menciones.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_usuarios

resultado = q3_time(file_path)
print(resultado)